In [4]:
%load_ext autoreload
%autoreload 2
from __future__ import absolute_import

import torch
import numpy as np
from perceptron.models.classification.pytorch import PyTorchModel
from perceptron.benchmarks.interval_analysis import NaiveIntervalMetric
from perceptron.benchmarks.interval_analysis import SymbolicIntervalMetric
from perceptron.utils.criteria.classification import Misclassification
from perceptron.utils.image import load_mnist_image
from perceptron.utils.tools import bcolors
from perceptron.zoo.mnist.model import mnist_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
param = {
  # epsilon for testing
  "epsilon" : 0.3,
  # epsilon for training
  "train_epsilon" : 0.3,
  # wheter need to check for optimal verified bound
  "optimal_bound" : True,
  # whether to parallelize on multiple gpus for testing
  "parallel" : True,
  # size of the model
  "model_size" : "large",
  # dataset for testing
  "dataset" : "mnist",
  # number of classes of testing dataset
  "num_classes" : 10,
  # bounds for testing dataset
  "bounds" : (0,1),
  # data format
  "data_format" : "channels_first",
  # mnist example image
  "fname" : "mnist0.png",
  # shape of images
  "shape" : (28,28)
}

image = load_mnist_image(shape=(28, 28), data_format='channels_first', fname='mnist0.png')

epsilon = param["epsilon"]

In [30]:
if param["dataset"] == "mnist":

  net = mnist_model(model_size=param["model_size"],
                    train_epsilon=param["train_epsilon"]).eval()

  param["num_classes"] = 10

if torch.cuda.is_available():
    net = net.cuda()

fmodel = PyTorchModel(
            net, bounds=param["bounds"], 
            num_classes=param["num_classes"]
          )
label = np.argmax(fmodel.predictions(image))

Load model from mnist_large_mixtrain_0.3.pth


In [38]:
from perceptron.benchmarks.carlini_wagner import CarliniWagnerLinfMetric
from perceptron.utils.criteria.classification import Misclassification
from perceptron.utils.distances import Linf
from perceptron.benchmarks.carlini_wagner import CarliniWagnerL2Metric

from perceptron.utils.distances import MSE
metric = CarliniWagnerL2Metric(fmodel, distance=MSE, criterion=Misclassification())

In [39]:
adversary = metric(image, label, unpack=False, binary_search_steps=10, max_iterations=2000)


100%|██████████| 10/10 [00:42<00:00,  6.57s/it]


new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3.47e-02
new best adversarial: normalized MSE = 3

In [33]:
print(adversary.distance)

normalized Linf distance = 9.17e-01


In [41]:
from perceptron.utils.tools import bound
ret = bound(image, model=fmodel, sd=0.1, num_class=10, num_iter=100)
print("{:.2e}".format(ret/784.))

7.65e-04
